In [14]:
# boiler plate shit 
#!git clone https://github.com/optmlclass/PA2.git
import torch, torchvision
import torchvision.transforms as transforms
from torch.optim import Optimizer
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from collections.abc import Iterable
import os
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

# make deterministic
from PA2.sequenceutils import set_seed
set_seed(42)

# imports for attention model
# more imports
from PA2.attentionmodel import GPT, GPTConfig
from PA2.sequenceutils import sample, CharDataset
from PA2.attentiontrainer import Trainer, TrainerConfig

os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
class AdamW_bias(Optimizer):
  def __init__(self, params, lr=1.0, betas=(0.9,0.999), weight_decay = 0.0, use_norm_scaling=False):
    super(AdamW_bias, self).__init__(params, {'lr': lr, 'beta1': betas[0], 'beta2': betas[1], 'weight_decay':weight_decay })


    epsilon = 1e-8

    self.use_norm_scaling = use_norm_scaling

    for group in self.param_groups:
      for p in group['params']:
        ## YOUR CODE HERE ##

        state = self.state[p]
        state['step'] = 0
        state['mt'] = torch.zeros_like(p, device=p.device) # p as param should work?
        state['vt'] = torch.zeros_like(p, device=p.device)

  @torch.no_grad()
  def step(self, closure=None):
    # in this class, and also usually in practice, closure will always be None.
    loss = None
    epsilon = 1e-8
    if closure is not None:
      with torch.enable_grad():
        loss = closure()
      
    ## YOUR CODE HERE ##
    for group in self.param_groups:
      lr = group['lr']
      beta1 = group['beta1']
      beta2 = group['beta2']
      weight_decay = group['weight_decay']
      
      for p in group['params']:
        if p.grad is None:
          continue
        state = self.state[p]
        state['step']+=1
        step = state['step']


        if self.use_norm_scaling: # aka 3 b
          # calcs all the relevant stuff
          gt = p.grad #+ weight_decay*p
          mt = (beta1*state['mt']) + ((1-beta1)*gt)
          vt = (beta2*state['vt']) + ((1-beta2)*torch.pow(gt,2))

          # updates new param/weight
          big_thingy = (mt/(torch.sqrt(vt)+epsilon))+weight_decay*p
          p.add_(big_thingy, alpha=(-lr*torch.norm(p)))

          # updates all the other stuff I forgot to 
          state['mt'] = mt
          state['vt'] = vt
          
        else: # aka 3 a 

          # calcs all the relevant stuff
          gt = p.grad #+ weight_decay*p #this stuff might not be good?
          mt = (beta1*state['mt']) + ((1-beta1)*gt)
          vt = (beta2*state['vt']) + ((1-beta2)*torch.pow(gt,2))
          mt_hat = mt/(1-pow(beta1,step))
          vt_hat = vt/(1-pow(beta2,step))

          # updates new param/weight
          big_thingy = (mt_hat/(torch.sqrt(vt_hat)+epsilon))+weight_decay*p
          p.add_(big_thingy, alpha=-lr)

          # updates all the other stuff I forgot to 
          #print('here is mt btw')
          #print(mt)
          state['mt'] = mt
          state['vt'] = vt
# generate training configurations for each of the optimizers. We will be testing
# adam (official pytorch implementation)
# adamw (official pytorch implementation)
# your optimizer both with and without the norm_scaling flag set.
def adamw_bias_factory(params, lr, betas):
  return AdamW_bias(params, lr, betas)

def adamw_bias_norm_scaling_factory(params, lr, betas):
  return AdamW_bias(params, lr, betas, use_norm_scaling=True,weight_decay=5.5e-2) # MATHEMATICALLY INFORMED 
# CHANGE BACK LATER 
optimizers = {
    'adamw_bias': adamw_bias_factory,
    'adamw_bias_norm_scaling': adamw_bias_norm_scaling_factory,
    'adam': torch.optim.Adam, 
    'adamw': torch.optim.AdamW
    #'adamw_bias': adamw_bias_factory,
    #'adamw_bias_norm_scaling': adamw_bias_norm_scaling_factory
  }

training_configs = {}
# the "block size" is the number of characters the model takes as input.
# in this case, it can look at up to 128 characters when predicting the next
# character.
block_size = 384 # spatial extent of the model for its context

# For our training set, we will use the text of the first four Harry Potter books.
text = open("/content/drive/MyDrive/henry_james.txt", 'rb').read()

train_dataset = CharDataset(text, block_size) # one line of poem is roughly 50 characters
for name, opt in optimizers.items():

  training_configs[name] = TrainerConfig(max_epochs=1, batch_size=256, learning_rate=6e-4, optimizer=opt,
                        lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset)*block_size,
                        num_workers=2)
torch.cuda.empty_cache()


cuda:0
data has 29343231 characters, 139 unique.


In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [13]:
import os
print(os.getcwd())

/content


In [15]:
# generate the configuration for the model. These parameters specify
# the neural network architecture we will be using. It is not necessary
# to understand this model architecture.
mconf_small = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=128)
mconf_med = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=10, n_head=10, n_embd=260)

In [ ]:
import gc
gc.collect()

3536

In [ ]:
## main training scripty :
best_optimizer = 'adamw_bias_norm_scaling'
long_train_conf = TrainerConfig(max_epochs=18, batch_size=32, learning_rate=5e-7, optimizer=optimizers[best_optimizer], # originally 6e-4
                        lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset)*block_size,
                        num_workers=2)
print("training new model with optimizer: {}".format(name))

train_dataset = CharDataset(text, block_size)
trainer = Trainer(best_model, train_dataset, None, long_train_conf)
train_loss = trainer.train()
print("final epoch train loss: {}".format(train_loss))

training new model with optimizer: adamw
data has 29343231 characters, 139 unique.


epoch 1 iter 2381: train loss 0.92529. lr 4.999692e-07: 100%|██████████| 2382/2382 [24:08<00:00,  1.64it/s]
epoch 2 iter 2381: train loss 0.91176. lr 4.998767e-07: 100%|██████████| 2382/2382 [21:21<00:00,  1.86it/s]
epoch 3 iter 2381: train loss 0.91183. lr 4.997225e-07: 100%|██████████| 2382/2382 [20:39<00:00,  1.92it/s]
epoch 4 iter 2381: train loss 0.94828. lr 4.995068e-07: 100%|██████████| 2382/2382 [20:42<00:00,  1.92it/s]
epoch 5 iter 2381: train loss 0.93823. lr 4.992294e-07: 100%|██████████| 2382/2382 [20:46<00:00,  1.91it/s]
epoch 6 iter 2381: train loss 0.92567. lr 4.988906e-07: 100%|██████████| 2382/2382 [20:45<00:00,  1.91it/s]
epoch 7 iter 2381: train loss 0.90179. lr 4.984904e-07: 100%|██████████| 2382/2382 [20:43<00:00,  1.92it/s]
epoch 8 iter 2381: train loss 0.90764. lr 4.980288e-07: 100%|██████████| 2382/2382 [20:44<00:00,  1.91it/s]
epoch 9 iter 2381: train loss 0.91088. lr 4.975061e-07: 100%|██████████| 2382/2382 [20:43<00:00,  1.91it/s]
epoch 10 iter 2381: train lo

final epoch train loss: 0.9183296606032227


In [17]:

context = [ord(c) for c in "I walked in on one brisk Thursday evening to my Henry James Class."]
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(device)
y = sample(best_model, x, 2000, temperature=0.9, sample=True, top_k=5)[0]
completion = ''.join([chr(train_dataset.itos[int(i)]) for i in y])
print(completion)

I walked in on one brisk Thursday evening to my Henry James Class.

I said to her that the Paris day was a confirmatory consideration, but that I had already been chattering a glance in the past took possession of her.  She was confidentially conscientious, but the sense that the past was a conclusion to the end of his confidence at any rate was to be confident to him that she could not have told him something of his conversation about the stage.  He was not of his own accepting the conversation, but he was surprised at the excursion that she was a good many years ago in the world; he was not sure that he should not have been so changing his career.  It was a stranger in this line that she had told him to be charmed and countries and a person.  She said nothing for anything, but she didnât say a word, âAre you going to tell him?â she asked, laughing.  âHe will say, in a way, anything but that he has got her away.â

She turned away, but her eyes were still strange.  âAre you

In [ ]:
#saves me fun model :)
torch.save(best_model.state_dict(),'henry_med.pth')

In [16]:
# loads the model back up for RETRAINING
PATH = '/content/drive/MyDrive/henry_med.pth'
best_model = GPT(mconf_med)
best_model.load_state_dict(torch.load(PATH))
best_model.to(device)

GPT(
  (tok_emb): Embedding(139, 260)
  (drop): Dropout(p=0.1, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((260,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((260,), eps=1e-05, elementwise_affine=True)
      (attn): CausalSelfAttention(
        (key): Linear(in_features=260, out_features=260, bias=True)
        (query): Linear(in_features=260, out_features=260, bias=True)
        (value): Linear(in_features=260, out_features=260, bias=True)
        (attn_drop): Dropout(p=0.1, inplace=False)
        (resid_drop): Dropout(p=0.1, inplace=False)
        (proj): Linear(in_features=260, out_features=260, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=260, out_features=1040, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=1040, out_features=260, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((260,), eps=1e-05, elementwise_a

In [ ]:
# loads up NEW model for training
best_model = GPT(mconf_med)

10/08/2022 23:00:59 - INFO - PA2.attentionmodel -   number of parameters: 8.318440e+06


In [ ]:
1e-2

0.01

In [ ]:
device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
print(device)

cpu
